# Sentencing

In [ ]:
# This Python file uses the following encoding: utf-8
import os
from lib.words import not_so_naive_split, replace_evil_dots_and_underscores, tokenize_doc
from lib.parallel_run import parallel_run, parallelize
import numpy as np
import codecs

Divisione del testo in frasi, rimozione di stop words, numeri, punteggiatura...

In [ ]:
# Esempio di split

t = not_so_naive_split(text, split='.')
#t

In [ ]:
# old, no utf-8...
def sentence_splitting(src_dir, out_dir, digit_replacement='NUM', split='.', min_words=2, max_n=-1):
    i = 0
    for f in os.listdir(src_dir):
        txt = open(src_dir+'/'+f).read()
        splitted = not_so_naive_split(txt, digit_replacement, split, min_words)
        with open(out_dir+'/'+f, 'w') as of:
            for l in splitted:
                of.write(' '.join(l)+'\n')
        if max_n>0 and i >= max_n:
            break
        i += 1
        
def sentence_tokenize_file(f, src_dir, out_dir, min_words=2):
    txt = open(src_dir+'/'+f).read()
    splitted = tokenize_doc(txt, min_words)
    with codecs.open(out_dir+'/'+f, 'w', 'utf-8') as of:
        for l in splitted:
            of.write(u' '.join(l)+u'\n')
        
def sentence_tokenization(src_dir, out_dir, min_words=2):
    parallelize(sentence_tokenize_file)(os.listdir(src_dir), src_dir, out_dir, min_words)

In [ ]:
# Per atti costitutivi
#sentence_splitting('../atti_costitutivi_txt','../atti_costitutivi_sentences')
sentence_tokenization('../atti_costitutivi_txt','../atti_costitutivi_sentences')

In [ ]:
# Per atti non costitutivi
#sentence_splitting('../atti_non_costitutivi_txt2','../atti_non_costitutivi_sentences')
sentence_tokenization('../atti_non_costitutivi_txt','../atti_non_costitutivi_sentences')

In [ ]:
# Per verbali
sentence_tokenization('../verbali_a05_txt/','../verbali_a05_sentences/')

In [ ]:
# Per atti non costitutivi scansionati
sentence_splitting('../ocr/scanned_non_costitutivi','../atti_non_costitutivi_sentences')

In [ ]:
# Esempio di testo elaborato

text = open('../atti_non_costitutivi_sentences/'+'4578843770001.txt').read()
print text

## Conversione in CSV

In [ ]:
def folders_to_csv(csv_file, dirs, labels):
    '''
    Writes to a csv file the lines contained in a list of directories,
    assuming each directory is associated to one label'''
    with codecs.open(csv_file,'w', 'utf-8') as out_file:
        out_file.write('filename,sentence,label\n')
        for d, label in zip(dirs, labels):
            for f in os.listdir(d):
                with codecs.open(d+u'/'+f, 'r', 'utf-8') as in_file:
                    for line in in_file:
                        out_file.write((u','.join([f[:-4],line.strip(),label]))+u'\n')
 

In [ ]:
folders_to_csv('../atti2.csv', ['../atti_costitutivi_sentences/','../atti_non_costitutivi_sentences/','../verbali_a05_sentences/'],['costitutivo','non_costitutivo','non_costitutivo'])

## Statistiche PANDAS

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def print_column_stats(col):
    print 'Mean', col.mean()
    print 'Stddev', col.std()
    print 'Min', col.min()
    print 'Max', col.max()

In [ ]:
df = pd.read_csv("../atti2.csv", encoding='utf-8')
#df

In [ ]:
df

In [ ]:
df_nc = df.loc[df['label'] == 'non_costitutivo']
df_c = df.loc[df['label'] == 'costitutivo']

## Distribuzione numero parole per linea

In [ ]:
s_len = df['sentence'].apply(lambda s:len(s.split()))

In [ ]:
print_column_stats(s_len)

In [ ]:
plt.figure(figsize=(20,10))
plt.hist([s for s in s_len if s<200], bins=100, normed=True) #[s for s in s_len if s<200]
#plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
s_len_c = df_c['sentence'].apply(lambda s:len(s.split()))
s_len_nc = df_nc['sentence'].apply(lambda s:len(s.split()))

In [ ]:
def hist_len(s_len):
    plt.figure(figsize=(20,10))
    plt.hist([s for s in s_len if s<200], bins=100, normed=True)
    #plt.yscale('log', nonposy='clip')
    plt.show()

In [ ]:
hist_len(s_len_c)
hist_len(s_len_nc)

## Distribuzione numero linee per file

In [ ]:
lines_per_file = df[['filename','sentence']].groupby(['filename']).size()

In [ ]:
print_column_stats(lines_per_file)

In [ ]:
plt.figure(figsize=(20,10))
plt.hist([l for l in lines_per_file if l<500], bins=300)
#plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
lines_per_file_nc = df_nc[['filename','sentence']].groupby(['filename']).size()
lines_per_file_c = df_c[['filename','sentence']].groupby(['filename']).size()

In [ ]:
print_column_stats(lines_per_file_c)
print_column_stats(lines_per_file_nc)

In [ ]:
def hist_lines(lines):
    plt.figure(figsize=(20,10))
    plt.hist([l for l in lines if l<300], bins=300)
    #plt.yscale('log', nonposy='clip')
    plt.show()

In [ ]:
hist_lines(lines_per_file_c)
hist_lines(lines_per_file_nc)

## NLTK test

In [ ]:
'''
import nltk
import re
from nltk.tokenize import WordPunctTokenizer
#tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')
word_tokenizer = WordPunctTokenizer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('italian'))
import string
punctuation = set(string.punctuation)
'''
from lib.words import allowed_lower_chars, tokenize_doc, read_codec_file, replace_evil_dots_and_underscores, contains_not_allowed_chars, replace_alnum, replace_num

In [ ]:
s = u'''ciao\n\n\n. eravamo in sei come va? questa è una super S.P.A. frase ART. perché affronterà;
allorchè poichè finchè èèèòòèòè beusjd fjkdhs §§§ 1 1223. molto  bene\n ciao\n muahah\n\n.'''

#for s in tokenize_doc(s,1):
 #   for w in s:
  #      print w

#print replace_evil_dots_and_underscores(s)
#print [l for l in u'fhdjkflasdààò fdàsaòùàòììì' if l in allowed_lower_chars]
print contains_not_allowed_chars(u'fhdjkflasdààòfdàsaòùàòììì')


for s in tokenize_doc(s, min_words=1):
    print u' '.join(s)

In [ ]:
sents =  tokenize_doc(read_codec_file('../atti_costitutivi_txt/4957502190001.txt'), min_words=2)
for s in sents:
    print ' '.join(s)


In [1]:
import nltk
import re
from nltk.tokenize import WordPunctTokenizer
import codecs
import json
import string
from lib import embedding as em
from nltk.corpus import stopwords
from lib.words import replace_evil_dots_and_underscores, splitted_words_utf8, replace_alnum, replace_num, clean_string_not_compressed
stop_words = set(stopwords.words('italian'))
punctuation = set(string.punctuation)
import roman
import operator

Using TensorFlow backend.


In [242]:
def index_matches(regex, lines, conv_ints=True):
    matches = (re.findall(regex, l) for l in lines)
    filt_matches = [(i,int(m[0]) if conv_ints else m[0]) for i,m in enumerate(matches) if m and len(m[0])>0]
    return filt_matches

def changes_only_iter(l, key_fn = lambda x:x):
    il = iter(l)
    t = next(il)
    yield t
    for i in il:
        if key_fn(i)!=key_fn(t):
            t = i
            yield i

def filter_matches_errors(ml):
    res = [ml[0]]
    for i in range(1, len(ml)-1):
        if (ml[i][1] == ml[i-1][1]+1) or (ml[i][1] == ml[i+1][1]-1):
            res.append(ml[i])
    res.append(ml[-1])
    return res

def matches_list(lines):
    num_regex = ur'(?u)^-*\s*(\d{1,2})[°\.\)](?!\d)'
    art_regex = r'(?i)^(?:- )?art\D{0,10}(\d{1,2})'
    roman_art_regex = r'(?i)^-*\s*?art\w*\.?\s+((?:ix)?(?:iv)?x?v?i{0,3})\s?'

    nm = index_matches(num_regex, lines)
    num_matches = list(changes_only_iter(nm, operator.itemgetter(1)))

    im = index_matches(art_regex, lines)

    imr_r = index_matches(roman_art_regex, lines, conv_ints=False)
    #print(imr_r)
    imr = [(i, roman.fromRoman(r)) for i,r in imr_r]

    matches = sorted(num_matches+im+ imr, key=operator.itemgetter(0))
    return matches
            
def end_statuto(lines, matches):
    found = False
    end_statuto_line = ''
    if len(matches) == 0:
        return ''
    im = iter(matches)
    _, t = next(im)
    for i, n in im:
        if found == True and n < t:
            end_statuto_line = lines[i]
            found = False
        if n < t:
            #print(n)
            found = True
        t = n
    return end_statuto_line

def end_statuto_text(text):
    lines = [l.strip() for l in text.split('\n') if len(l.strip())>0]
    matches = matches_list(lines)
    corrected_matches = filter_matches_errors(matches)
    return end_statuto(lines, corrected_matches)

In [250]:
#text = codecs.open('../atti_costitutivi_txt/4957502190001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4909480860001.txt', encoding="utf-8").read()
text = codecs.open('../statuti/molesti/4908140980001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4907851720001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4907874650001.txt', encoding="utf-8").read() #roman
#text = codecs.open('../statuti/molesti/roman_stat_in_mezzo.txt', encoding="utf-8").read() #roman inside
#text = codecs.open('../statuti/molesti/schifo.txt', encoding="utf-8").read()


In [251]:
lines = [l.strip() for l in text.split('\n') if len(l.strip())>0]

In [252]:
list(enumerate(lines))

[(0,
  u'Copia su supporto informatico conforme al documento originale formato su'),
 (1,
  u"supporto cartaceo, ai sensi dell'articolo 22 del D.LGS. n. 82/2005, che si"),
 (2, u'trasmette ad uso Registro Imprese.'),
 (3,
  u'Imposta di bollo assolta ai sensi del decreto 22/02/2007 mediante M.U.I.'),
 (4, u'Repertorio n. 66799'),
 (5, u'ATTO COSTITUTIVO'),
 (6, u'LIMITATA'),
 (7, u'DI'),
 (8, u'Raccolta n. 32967'),
 (9, u"SOCIETA' A RESPONSABILITA'"),
 (10, u'Repubblica Italiana'),
 (11, u"L'anno duemiladiciassette, il giorno uno del mese di febbraio."),
 (12, u'In Azzano Decimo, nel mio studio in via Maestri del Lavoro n. 12/1.'),
 (13, u'Avanti a me dottor Maria Luisa Sperandeo, Notaio in Azzano'),
 (14, u'Decimo, iscritta nel Collegio Notarile del Distretto di Pordenone'),
 (15, u'sono presenti i signori'),
 (16,
  u'- LUCIANO Massimo, nato a Pordenone (PN) il 7 agosto 1980, residente a'),
 (17, u'San Vito al Tagliamento (PN), via Prodolone n. 14/A,'),
 (18, u'Codice Fiscale LCN MSM

In [253]:
end_statuto_text(text)

u"Art. 9) Si d\xe0 atto che l'importo globale approssimativo delle spese per"